# Mask R-CNN - Squirrel Monkey Segmentation


In [1]:
# Gets rid of a HOST of deprecation warnings for Matterport
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")


# and Tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [2]:
from mrcnn.model import log
from mrcnn import visualize
import mrcnn.model as modellib
from mrcnn import utils
from mrcnn.config import Config
import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import skimage
from termcolor import colored

# Root directory of the project
ROOT_DIR = os.path.abspath("./Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# COCO_MODEL_PATH = "C:\\Users\\addis\\Documents\\mask_rcnn_coco.h5"
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)


Using TensorFlow backend.


In [3]:
# Check tf version
print(tf.__version__)
print(tf.test.is_gpu_available())


1.15.3
True


## Configurations


In [4]:
class MonkeysConfig(Config):
    #################### BASE CONFIGURATION ####################
    NAME = "monkeys"

    # Train on 1 GPU, with batch size 4
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # BATCH_SIZE = 4

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 monkey
    DETECTION_MAX_INSTANCES = 1  # we're only looking for the most prominent individual in each image

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # RPN ANCHOR SCALES left as default (32, 64, 128, 256, 512), in line with the FaterRCNN paper
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # (the number of batch iterations before a training epoch is considered finished). As we want to train on the full dataset, it's equal to num_samples/batch_size
    # STEPS_PER_EPOCH = 100

    # VALIDATION_STEPS is similiar to STEPS_PER_EPOCH


class InferenceConfig(MonkeysConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


def get_config(learning_rate, image_dim, detection_nms_threshold, detection_min_confidence, num_samples, batch_size):
    # config used for both training and inference
    train_config = MonkeysConfig()
    inference_config = InferenceConfig()

    for config in [train_config, inference_config]:
        #################### HYPERPARAMETERS TO TUNE ####################
        config.IMAGE_MAX_DIM = image_dim
        config.IMAGE_MIN_DIM = image_dim
        config.IMAGE_SHAPE = np.array([image_dim, image_dim, 3])

        config.LEARNING_RATE = learning_rate

        config.DETECTION_NMS_THRESHOLD = detection_nms_threshold

        config.DETECTION_MIN_CONFIDENCE = detection_min_confidence

        config.STEPS_PER_EPOCH = int(num_samples / batch_size)

    train_config.display()

    return train_config, inference_config


## Dataset

Handles loading images and masks for the custom dataset


In [5]:
import json
MONKEY_CLASS_ID_STR = "monkey"


class MonkeysDataset(utils.Dataset):
    def load_monkeys(self, dataset_dir, subset):

        # Add classes
        self.add_class(MONKEY_CLASS_ID_STR, 1, MONKEY_CLASS_ID_STR)

        num_images_added = 0
       # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(
            open(os.path.join(dataset_dir, "via_region_data.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            if a['filename'] in subset:
                # Get the x, y coordinaets of points of the polygons that make up
                # the outline of each object instance. These are stores in the
                # shape_attributes (see json format above)
                # The if condition is needed to support VIA versions 1.x and 2.x.
                if type(a['regions']) is dict:
                    polygons = [r['shape_attributes']
                                for r in a['regions'].values()]
                else:
                    polygons = [r['shape_attributes'] for r in a['regions']]

                # load_mask() needs the image size to convert polygons to masks.
                # Unfortunately, VIA doesn't include it in JSON, so we must read
                # the image. This is only managable since the dataset is tiny.
                image_path = os.path.join(dataset_dir, "images", a['filename'])
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]

                self.add_image(
                    MONKEY_CLASS_ID_STR,
                    image_id=a['filename'],  # use file name as a unique image id
                    path=image_path,
                    width=width, height=height,
                    polygons=polygons)

                num_images_added += 1
                print(colored(f"Loading images {num_images_added}/{len(annotations)}"), end='\r')

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a monkey dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != MONKEY_CLASS_ID_STR:
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == MONKEY_CLASS_ID_STR:
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


# Loading Dataset

Ensure the dataset is in the following form:

dirName  
└── train  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── a.jpg  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── b.jpg  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── c.jpg  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── via_region_data.json  
└── val  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── c.jpg  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── d.jpg  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── e.jpg  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── via_regon_data.json


In [6]:
import glob
from pathlib import Path
DATASET_DIR = Path("F:/Adam/Pictures/AucklandZooImages/cv set (240)")

path = (DATASET_DIR / "images" / "*.*")

X = [os.path.basename(x) for x in glob.glob(str(path))]
print(X[:3])


['IMG_1718_00108006.jpg', 'IMG_1718_00108010.jpg', 'IMG_1718_00108017.jpg']


## K-Fold Validation


In [7]:
def prepare_model_train_dataset(X_train_fold):
    print("Preparing Dataset")
    # Annotated using: https://www.robots.ox.ac.uk/~vgg/software/via/
    dataset = MonkeysDataset()
    dataset.load_monkeys(DATASET_DIR, subset=X_train_fold)
    dataset.prepare()

    return dataset


In [8]:
def prepare_train_model(model_configuration):
    print("Preparing training model...")

    # Create model in training mode
    model = modellib.MaskRCNN(mode="training", config=model_configuration,
                              model_dir=MODEL_DIR)

    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])

    return model


In [9]:
def train_model(dataset_train, dataset_val, train_config, training_epochs, fine_tune_epochs):

    model = prepare_train_model(train_config)

    print(colored("*"*30 + "Training Model" + "*"*30, 'green'))
    

    print("\n\n")
    print("*"*30, " Training model head layers ", "*"*30)
    print("\n\n")

    # Train the head branches
    # Passing layers="heads" freezes all layers except the head
    # layers. You can also pass a regular expression to select
    # which layers to train by name pattern.

    model.train(dataset_train, dataset_val,
                learning_rate=train_config.LEARNING_RATE,
                epochs=training_epochs,
                layers='heads')

    print("\n\n")
    print("*"*30, " Fine tune all layers ", "*"*30)
    print("\n\n")

    # Fine tune all layers
    # Passing layers="all" trains all layers. You can also
    # pass a regular expression to select which layers to
    # train by name pattern.
    model.train(dataset_train, dataset_val,
                learning_rate=train_config.LEARNING_RATE / 10,  # TODO determine why this is / 10
                epochs=training_epochs+fine_tune_epochs,
                layers="all")


In [10]:
def get_inference_model(inference_config):
    print("Getting inference model")
    # Recreate the model in inference mode
    model = modellib.MaskRCNN(mode="inference",
                              config=inference_config,
                              model_dir=MODEL_DIR)

    # Get path to saved weights
    # Either set a specific path or find last trained weights
    # model_path = os.path.join(ROOT_DIR, ".h5 file name here")
    model_path = model.find_last()

    # Load trained weights
    print("Loading weights from ", model_path)
    model.load_weights(model_path, by_name=True)

    return model


def evaluate_model(dataset_val, inference_config):
    print("\n\n")
    print("*"*30, " Evaluating model ", "*"*30)
    print("\n\n")

    # Compute VOC-Style mAP @ IoU=0.5
    # Running on 10 images. Increase for better accuracy.
    # image_ids = np.random.choice(dataset_val.image_ids, 10)

    image_ids = dataset_val.image_ids

    model = get_inference_model(inference_config)

    APs = []
    for i, image_id in enumerate(image_ids):
        print(f"Evaluating image {i}/{len(image_ids)}", end='\r')

        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, inference_config,
                                   image_id, use_mini_mask=False)
        molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        # Compute AP
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold = IOU_THRESHOLD)
        APs.append(AP)

    print("Finished evaluating")
    return np.mean(APs)


In [11]:
from sklearn.model_selection import train_test_split


def kfold_model(n_splits, X_train, model_config, inference_config, train_epochs, fine_tune_epochs):

    kf = KFold(n_splits=n_splits, random_state=SEED, shuffle=True)

    mAPs = []

    count = 1

    for train_index, test_index in kf.split(X_train):
        X_train = np.array(X_train)
        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]

        print(colored("*"*30 + f" Beginning fold {count} " + "*"*30, 'green'))
        count += 1

        # Split our training data further into a train and validation set that will be used during *Training*
        X_train_train_sub, X_train_val_sub = train_test_split(X_train_fold, test_size=0.1, random_state=SEED)

        # Train and validation sets used during model training
        dataset_train = prepare_model_train_dataset(X_train_train_sub)
        dataset_val = prepare_model_train_dataset(X_train_val_sub)

        # Test set used to evaluate model performance *Testing*
        dataset_test = prepare_model_train_dataset(X_test_fold)

        train_model(dataset_train, dataset_val, model_config, train_epochs, fine_tune_epochs)

        # Mean Average Precision for the trained model
        mAP = evaluate_model(dataset_test, inference_config)

        mAPs.append(mAP)

    # Averaged mAP accross the k folds
    averaged_mAPs = np.mean(mAPs)
    print(colored(f"mAP: {averaged_mAPs}"))

    return averaged_mAPs


In [12]:
from itertools import product
from sklearn.model_selection import KFold


def test_hyperparameters(num_folds, X_train, train_epochs, fine_tune_epochs):
    learning_rate_search_space = [0.01, 0.001, 0.0001]
    image_dim_search_space = [1024, 512, 256]
    detection_nms_search_space = [0.2, 0.3, 0.4]
    detection_min_confidence_search_space = [0.7, 0.8, 0.9]

    num_hyperparameters = 4

    search_permutations = list(product(learning_rate_search_space, image_dim_search_space, detection_nms_search_space, detection_min_confidence_search_space))

    results = np.zeros((len(search_permutations), num_hyperparameters + 1))

    for i, combination in enumerate(search_permutations):
        learning_rate, image_dim, detection_nms_threshold, detection_min_confidence = combination

        print("\n\n")
        print(colored("*"*30 + f" Evaluating variation {i+1}/{len(search_permutations)} " + "*"*30, "green"))
        print("\n\n")

        # Model configurations, with hyperparameters
        train_config, inference_config = get_config(learning_rate, image_dim, detection_nms_threshold, detection_min_confidence, num_samples=len(X_train), batch_size=1)

        averaged_mAPs = kfold_model(num_folds, X_train, train_config, inference_config, train_epochs, fine_tune_epochs)

        results[i, :] = learning_rate, image_dim, detection_nms_threshold, detection_min_confidence, averaged_mAPs

    return results


In [13]:

SEED = 123

X_train, X_val = train_test_split(X, test_size=0.2, random_state=SEED)

print(f"X_train length: {len(X_train)}")
print(f"Reservered X_val length: {len(X_val)}")


X_train length: 50
Reservered X_val length: 10


In [14]:
num_folds = 2
num_train_epochs = 1
num_fine_tune_epochs = 1

results = test_hyperparameters(num_folds, X_train, num_train_epochs, num_fine_tune_epochs)





****************************** Evaluating variation 1/81 ******************************




Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.2
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.01
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bb

ResourceExhaustedError: OOM when allocating tensor with shape[1,512,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node training_5/SGD/gradients/rpn_model_20/rpn_bbox_pred/convolution_grad/Conv2DBackpropInput (defined at E:\Conda\envs\MaskRCNN\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'training_5/SGD/gradients/rpn_model_20/rpn_bbox_pred/convolution_grad/Conv2DBackpropInput':
  File "c:\Users\Adam\.vscode\extensions\ms-toolsai.jupyter-2021.9.1101343141\pythonFiles\vscode_datascience_helpers\kernel_prewarm_starter.py", line 31, in <module>
    runpy.run_module(module, run_name="__main__", alter_sys=False)
  File "E:\Conda\envs\MaskRCNN\lib\runpy.py", line 208, in run_module
    return _run_code(code, {}, init_globals, run_name, mod_spe